<a href="https://www.kaggle.com/code/nicholastadeu/nps-analysis-project?scriptVersionId=144611468" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
#importing libraries

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [ ]:
#reading the csv file as a pandas dataframe
nps_data = pd.read_csv('/kaggle/input/npsbank/NPStimeseries.csv')

In [ ]:
#analizyng the df head

nps_data.head()

In [ ]:
#analizyng each collum

nps_data.info()

In [ ]:
#analizyng the shape

nps_data.shape

In [ ]:
#visualizing the df

nps_data

In [ ]:
# counting the unique values in the clients column and showing the max and minimum times that one customer reponded the survey
nps_data['Customer Name'].value_counts()


In [ ]:
#counting the unique values in the NPS Score column and showing the max and minimum times that the score was choosed
nps_data['NPS'].value_counts()

In [ ]:
# specifying the column names that I want to use

new_cols_dict ={
    'Survey date':'Date',
    'Customer Name':'Name',
    'NPS':'Score'
}
        
#renaming the columns to the specified column names
nps_data.rename(new_cols_dict, axis=1, inplace=True)

nps_data

In [ ]:
#creating a new collumn with the NPS profile of each client, to facilitate calculating NPS
def define_profile(score):
    if score <= 6:
        return 'Detractor'
    elif score <= 8:
        return 'Passive'
    else:
        return 'Promoter'

#apllying the function to create the new "Profile" column
nps_data['Profile'] = nps_data['Score'].apply(define_profile)

In [ ]:
#making sure that the changes are effective

nps_data.head()

In [ ]:
#ploting a graphic with the % of each nps profile per market

#calculating the % of each profile by market
profile_percentage = nps_data.groupby(['Market', 'Profile']).size() / nps_data.groupby('Market').size() * 100
profile_percentage = profile_percentage.unstack().fillna(0)

#defining colors for profiles
colors = {'Promoter': 'green', 'Passive': 'sandybrown', 'Detractor': 'red'}

#ploting the stacked bar chart
ax = profile_percentage.plot(kind='bar', stacked=True, color=[colors[col] for col in profile_percentage.columns])

plt.xlabel('Market')
plt.ylabel('Percentage')
plt.title('Percentage of Profiles by Market')

#displaying legend with custom labels
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=[plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels], labels=labels)

#adding labels with percentage values on top of the bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.2f%%', label_type='center', fontsize=10, color='white')
    
plt.ylim(0, 150)


plt.show()

In [ ]:
#plting a graphic to see how many responses each market has

#group by Market and count the responses
responses_count = nps_data.groupby('Market')['Score'].count()


#ploting a bar chart
ax = responses_count.plot(kind='bar', color='blue', edgecolor='black')

#adding labels with the number of responses above each bar
for i, v in enumerate(responses_count):
    ax.text(i, v + 1, str(v), ha='center', va='bottom', fontsize=12)
    
plt.ylim(0, 2000)


#setting labels, title, and legend
plt.xlabel('Market')
plt.ylabel('Number of Responses')
plt.title('Number of Responses by Market')
plt.legend(['Responses'])

plt.show()

In [ ]:
#answering the first big question, "what is the NPS Score of my base?"

#calculating the total count of Promoters, Neutrals, and Detractors
total_count = len(nps_data)
promoter_count = (nps_data['Profile'] == 'Promoter').sum()
passive_count = (nps_data['Profile'] == 'Passive').sum()
detractor_count = (nps_data['Profile'] == 'Detractor').sum()

promoter_porcentage = promoter_count / total_count * 100
passive_porcentage = passive_count / total_count * 100
detractor_porcentage = detractor_count / total_count * 100

#calculating the overall NPS (Promoters - Detractors) for the entire DataFrame
overall_nps = (promoter_count - detractor_count) / total_count * 100

#printing the percentage and the final grade results. I'm printing the percentages too as a prove that the final grade is correct
print(f" Promoters: {promoter_porcentage:.2f}")
print(f" Passives: {passive_porcentage:.2f}")
print(f" Detractors: {detractor_porcentage:.2f}")

print(f" NPS Score: {overall_nps:.2f}")


In [ ]:
#aswering the secund big question, "Witch Market has the best NPS?" by calculating NPS per market

#calulating the total os responses per market
market_counts = nps_data['Market'].value_counts()

#counting promoters and detractors by market
promoter_counts = nps_data[nps_data['Profile'] == 'Promoter']['Market'].value_counts()
detractor_counts = nps_data[nps_data['Profile'] == 'Detractor']['Market'].value_counts()

#calculating the % of promoters and detractors per market
percentage_promoters = (promoter_counts / market_counts) * 100
percentage_detectors = (detractor_counts / market_counts) * 100

#calculating the NPS (% of Promoters - % of Detractors) per market
nps_by_market = percentage_promoters - percentage_detectors

#creating a new df to put the results
nps_result_by_market = pd.DataFrame({'Market': nps_by_market.index, 'NPS': nps_by_market.values})

#printing the result
nps_result_by_market

In [ ]:
#checking if the original df wasn't affected
nps_data

In [ ]:
#calculating NPS per day, this will help to validate the data on tableau afterwards

#creating a function to calculate the nps
def calculate_nps(group):
    promoters = (group['Profile'] == 'Promoter').sum()
    detractors = (group['Profile'] == 'Detractor').sum()
    total = len(group)
    return ((promoters - detractors) / total) * 100

#grouping all the responses per date in a new df
nps_by_date = nps_data.groupby('Date').apply(calculate_nps).reset_index(name='NPS')

# Exiba o resultado
print(nps_by_date)


In [ ]:
#choosing a name for the archive
output_excel_file = 'nps_analisys.xlsx'

#exporting the df to an excel file
nps_data.to_excel(output_excel_file, index=False)

# **Now that we have our data for tableu, i'll star a process more focused on data science in our df**

In [ ]:
#making sure that the column Date is actually datetime
nps_data['Date'] = pd.to_datetime(nps_data['Date'], format='%d/%m/%Y')

#creating a new df with only the columns I will use
nps_asis = nps_data[['Date', 'Score', 'Profile']].copy()

#adding a new column, with the week day of the date
nps_asis['Day_of_Week'] = nps_data['Date'].dt.day_name()

#seeing the new df
nps_asis

In [ ]:
#counting the answers per week day
count_by_day_of_week = nps_asis['Day_of_Week'].value_counts().reset_index()

#renaming the columns
count_by_day_of_week.columns = ['Day_of_Week', 'Count']

count_by_day_of_week = count_by_day_of_week.sort_values(by='Count', ascending=False)


#seeing the result
print(count_by_day_of_week)

In [ ]:
#creating a function to calculate the nps
def calculate_nps(group):
    promoters = (group['Profile'] == 'Promoter').sum()
    detractors = (group['Profile'] == 'Detractor').sum()
    total = len(group)
    nps = ((promoters - detractors) / total) * 100
    return round(nps, 2) 

#grouping by Day of week and calculating the nps
nps_by_day_of_week = nps_asis.groupby('Day_of_Week').apply(calculate_nps).reset_index(name='NPS')

#ordering from biggest to smallest
nps_by_day_of_week = nps_by_day_of_week.sort_values(by='NPS', ascending=False)

#seeing the result
nps_by_day_of_week


In [ ]:
from scipy.stats import chi2_contingency

#creating a crosstab between Score and Profile
contingency_table = pd.crosstab(nps_asis['Score'], nps_asis['Profile'])

#doing the test
chi2, p, _, _ = chi2_contingency(contingency_table)

#showing the p-value
print(f"P-Value: {p}")

#analyzing the result
alpha = 0.05 
if p <= alpha:
    print("There is a correlation between Score and Profile")
else:
    print("The correlation doesen't exists")


In [ ]:
import statsmodels.api as sm

#creating a column Days_Since_Start that represents the counting of days starting at a refer date
nps_asis['Date'] = pd.to_datetime(nps_asis['Date'])  
start_date = min(nps_asis['Date']) 
nps_asis['Days_Since_Start'] = (nps_asis['Date'] - start_date).dt.days

#adding a constant for the regression
nps_asis['const'] = 1

#defining a independent and dependent variables
X = nps_asis[['const', 'Days_Since_Start']]
y = nps_asis['Score']

#running the model
model = sm.OLS(y, X).fit()

#analyzing the result
model.summary()


In general, the regression results indicate that the variable **'Days_Since_Start'** is not significantly related to customer scores. **The low R-squared** suggests that other factors not included in the model may play a more significant role in explaining variations in customer scores.

Therefore, let's now focus more on the days of the week rather than the open date.

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols


#creating a new df for the analysis
analysis_df = nps_asis[['Day_of_Week', 'Score']]

#performing an Analysis of Variance (ANOVA)
model = ols('Score ~ Day_of_Week', data=analysis_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

#analyzing the result
print(anova_table)
#if the p-value (PR(>F)) is less than a chosen significance level (e.g. 0.05), you can conclude that there is a significant correlation between 'Day_of_Week' and 'Score'.

Based on the analysis of variance, there does not appear to be a significant correlation between the day of the week (column 'Day_of_Week') and the scores (column 'Score'). Therefore, there is no statistical evidence to claim that the days of the week have a significant impact on customer scores based on the analyzed data.

# Now, I will respond the last big question "The clients that responded more than one time, the perception of value improved or not?"

In [ ]:
#selecting only the occurrences of "Name" that are duplicated
repeated_names = nps_data[nps_data['Name'].duplicated(keep=False)]

#creating a new DataFrame with the duplicated occurrences
nps_timeline = pd.DataFrame(repeated_names)

#reseting the index of the new DataFrame, if necessary
nps_timeline.reset_index(drop=True, inplace=True)

#analysing the result
nps_timeline


In [ ]:
#making sure that there are only duplicates
nps_timeline['Name'].value_counts()

In [ ]:

#sorting the DataFrame by 'Name' and 'Date'
nps_timeline.sort_values(by=['Name', 'Date'], inplace=True)

#creating a new DataFrame to store the timeline
timeline_df = pd.DataFrame(columns=['Name', 'Profile_From', 'Profile_To', 'Date_From', 'Date_To'])

#initialyzing variables to track the profile transitions
current_name = None
profile_from = None
date_from = None

#iterating through the sorted DataFrame
for index, row in nps_timeline.iterrows():
    if current_name is None:
        current_name = row['Name']
        profile_from = row['Profile']
        date_from = row['Date']
    elif current_name == row['Name']:
        if profile_from != row['Profile']:
            # Profile transition detected, record the change in the timeline DataFrame
            timeline_df = pd.concat([timeline_df, pd.DataFrame({'Name': [current_name],
                                                                'Profile_From': [profile_from],
                                                                'Profile_To': [row['Profile']],
                                                                'Date_From': [date_from],
                                                                'Date_To': [row['Date']]})], ignore_index=True)
            profile_from = row['Profile']
            date_from = row['Date']
    else:
        current_name = row['Name']
        profile_from = row['Profile']
        date_from = row['Date']

#cheking if the last profile transition is missing and add it to the timeline
if current_name is not None:
    timeline_df = pd.concat([timeline_df, pd.DataFrame({'Name': [current_name],
                                                        'Profile_From': [profile_from],
                                                        'Profile_To': [profile_from],  # Use the last profile as 'Profile_To'
                                                        'Date_From': [date_from],
                                                        'Date_To': [nps_timeline['Date'].max()]})], ignore_index=True)

#analysing the result
timeline_df

In [ ]:
#counting occurrences of transitions from Detractor to Promoter
det_to_prom_count = (timeline_df[(timeline_df['Profile_From'] == 'Detractor') & (timeline_df['Profile_To'] == 'Promoter')]
                     .shape[0])

#counting occurrences of transitions from Promoter to Detractor
prom_to_det_count = (timeline_df[(timeline_df['Profile_From'] == 'Promoter') & (timeline_df['Profile_To'] == 'Detractor')]
                     .shape[0])

#counting occurrences of transitions from Neutrals to Promoters
neu_to_prom_count = (timeline_df[(timeline_df['Profile_From'] == 'Passive') & (timeline_df['Profile_To'] == 'Promoter')]
                     .shape[0])

#counting occurrences of transitions from Neutrals to Detractors
neu_to_det_count = (timeline_df[(timeline_df['Profile_From'] == 'Passive') & (timeline_df['Profile_To'] == 'Detractor')]
                     .shape[0])

print("Occurrences of transitions:")
print("Detractor to Promoter:", det_to_prom_count)
print("Promoter to Detractor:", prom_to_det_count)
print("Passive to Promoters:", neu_to_prom_count)
print("Passive to Detractors:", neu_to_det_count)


Overall, these transitions highlight the dynamic nature of customer sentiment and the potential for both positive and negative shifts in customer perception over time. It's essential for businesses to monitor and understand these transitions to improve customer satisfaction and loyalty.